In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from threading import Thread
from queue import Queue
import multiprocessing

In [3]:
xcelLocation = "../xcels/"
!export xcelLocation="../xcels/"

In [4]:
!ls $xcelLocation | grep ".xlsx" > xlFiles
tmp = !cat xlFiles
names = [name[:-5] for name in tmp]

In [5]:
def cleaner():
    for name in names:
        if os.path.getsize(xcelLocation + name + '.xlsx') < 10000:
            os.remove(xcelLocation + name + '.xlsx')
cleaner()

In [6]:
def convert(xcelLocation, xlFileName, returnAllMode=False):
    xl = None
    try:
        if (not Path(xcelLocation + xlFileName + '.csv').is_file()) or returnAllMode:
            xl = pd.read_excel(xcelLocation + xlFileName + ".xlsx", index_col=0,
                               header=[0], skiprows=[0,1])
            xl.to_csv(xcelLocation + xlFileName + '.csv', encoding='utf-8', index=False)
    except:
        xl = str(xlFileName)
    finally:
        return xl

In [7]:
def convertThread(threadname, q, qDFs, qErrors):
    while not q.empty():
        fileNames = q.get()
        q.task_done()
        for name in fileNames:
            tmp = convert(xcelLocation=xcelLocation, xlFileName=name, returnAllMode=True)
            if isinstance(tmp, str):
                qErrors.put(tmp)
            else:
                qDFs.put((tmp.copy(), name))
    print(str(threadname) + " done")


In [ ]:
allDFs = []
allDFNames = []
batchSize = 10
i = 0
numThread = 16
workers = []

pool = multiprocessing.Pool(processes=numThread)
m = multiprocessing.Manager()
queue = m.Queue()
qDFs = m.Queue()
qErrors = m.Queue()

while i*batchSize < len(names):
    if (i+1)*batchSize < len(names):
        queue.put(names[i*batchSize:(i+1)*batchSize])
    else:
        queue.put(names[i*10:])
    i+=1
print(len(names))
print(queue.qsize())

for i in range(numThread):
#     workers.append(Thread(target=readThread, args=("Thread-" + str(i), queue, qsum, qcount)))
#     workers.append(pool.apply_async(readThread, ("Thread-" + str(i), queue, qsum, qcount,)))
    workers.append(multiprocessing.Process(target=convertThread, args=("Thread-" + str(i),
                                                                    queue, qDFs, qErrors)))
    workers[i].start()

for i in range(numThread):
    workers[i].join()

while not qDFs.empty():
    dftmp, nametmp = qDFs.get()
    allDFs.append(dftmp)
    allDFNames.append(nametmp)

4410
441
Thread-7 done
Thread-2 done
Thread-3 done
Thread-9 done
Thread-12 done
Thread-13 done
Thread-15 done
Thread-11 done
Thread-8 done
Thread-1 done
Thread-6 done
Thread-10 done
Thread-4 done
Thread-0 done
Thread-5 done
Thread-14 done


In [ ]:
print(len(allDFs))
def makeMasterTable():
    for index, df in enumerate(allDFs):
        year, month, day = allDFNames[index].split("-")
        yearlist = np.full(len(df), year).tolist()
        monthlist = np.full(len(df), month).tolist()
        daylist = np.full(len(df), day).tolist()
        df["year"] = yearlist
        df["month"] = monthlist
        df["day"] = daylist
    xl = pd.concat(allDFs, keys=allDFNames, ignore_index=True)
    xl.to_csv(xcelLocation + 'master.csv', encoding='utf-8', index=False)
    return xl
allDF = makeMasterTable()

In [ ]:
allDF.loc[allDF["نام"] == "شير و گوشت زاگرس شهركرد"]